# B3D Electric Field File I/O

Demonstrates creating, writing, reading, and validating B3D binary files for
electric field data used in PowerWorld GIC analysis.

Topics covered:
- Creating B3D objects from scratch
- Building B3D from mesh-grid data with `from_mesh()`
- Writing and reading B3D files
- Round-trip verification
- Visualizing E-field data from B3D files

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from esapp.utils import B3D, format_plot, border

In [ ]:
# Plotting functions (hidden from documentation)
import sys; sys.path.insert(0, "..")
from plot_helpers import plot_b3d_components, plot_b3d_roundtrip

## 1. Creating a B3D Object from Scratch

The `B3D` class stores time-varying electric field data at geographic locations.

In [ ]:
b3d = B3D()

print(f"Default B3D object:")
print(f"  Comment: {b3d.comment}")
print(f"  Grid dimensions: {b3d.grid_dim}")
print(f"  Locations: {len(b3d.lat)}")
print(f"  Time steps: {len(b3d.time)}")
print(f"  Lat: {b3d.lat}")
print(f"  Lon: {b3d.lon}")
print(f"  Ex shape: {b3d.ex.shape}")
print(f"  Ey shape: {b3d.ey.shape}")

## 2. Building B3D from Mesh-Grid Data

Use `B3D.from_mesh()` to construct a B3D from regularly-spaced geographic arrays.
This is the most common workflow for custom E-field creation.

In [ ]:
# Define geographic grid covering Texas
nx, ny = 25, 20
lons = np.linspace(-106, -93, nx)
lats = np.linspace(25.5, 36.5, ny)
LON, LAT = np.meshgrid(lons, lats)

# Create a spatially-varying E-field: gaussian hot spot
lon_c, lat_c = -99.5, 31.0
sigma = 2.0
gaussian = np.exp(-((LON - lon_c)**2 + (LAT - lat_c)**2) / (2 * sigma**2))

Ex = 2.0 * gaussian  # V/km eastward
Ey = 0.5 * gaussian  # V/km northward

b3d = B3D.from_mesh(
    long=lons, lat=lats, ex=Ex, ey=Ey,
    comment="Gaussian hotspot E-field over Texas"
)

print(f"B3D from mesh:")
print(f"  Grid: {b3d.grid_dim}")
print(f"  Points: {len(b3d.lat)}")
print(f"  Ex range: [{b3d.ex.min():.3f}, {b3d.ex.max():.3f}] V/km")
print(f"  Ey range: [{b3d.ey.min():.3f}, {b3d.ey.max():.3f}] V/km")

## 3. Visualizing the E-Field

In [ ]:
plot_b3d_components(LON, LAT, Ex, Ey, 'Texas',
                    suptitle='Gaussian Hotspot E-Field')

## 4. Write and Read Round-Trip

Write the B3D to disk and read it back to verify data integrity.

In [ ]:
# Write
b3d.write_b3d_file("gaussian_efield.b3d")
print("Written to gaussian_efield.b3d")

# Read back
b3d_loaded = B3D("gaussian_efield.b3d")

print(f"\nRound-trip verification:")
print(f"  Comment: '{b3d_loaded.comment}'")
print(f"  Grid dim: {b3d_loaded.grid_dim}")
print(f"  Lat error: {np.max(np.abs(b3d_loaded.lat - b3d.lat)):.2e}")
print(f"  Lon error: {np.max(np.abs(b3d_loaded.lon - b3d.lon)):.2e}")
print(f"  Ex error:  {np.max(np.abs(b3d_loaded.ex - b3d.ex)):.2e}")
print(f"  Ey error:  {np.max(np.abs(b3d_loaded.ey - b3d.ey)):.2e}")

In [ ]:
plot_b3d_roundtrip(LON, LAT, b3d.ex, b3d_loaded.ex, 'Texas', ny, nx)

## Summary

The B3D format provides a compact binary representation for spatially-varying
electric field data:
- Use `B3D.from_mesh()` to create from lon/lat/Ex/Ey arrays
- Use `write_b3d_file()` to serialize to disk
- Use `B3D(filename)` to load
- Load into PowerWorld with `wb.gic.loadb3d()` for non-uniform GIC analysis